imports

In [ ]:
# imports
import pandas as pd
import numpy as np
import json

loading data

In [ ]:
# location dataframe
location_df = pd.read_csv('SaO_Optilandia_resub_locations.csv')

# links dataframe
links_df = pd.read_csv('SaO_Optilandia_resub_links.csv')

# extract lorry data from json
lorry_data = json.load(open('SaO_Optilandia_resub_depot_lorries.json', 'r'))

# set count to 0
k = 0

# initialise lorry list
lorry = []

# loop -> set i to the respective lorry key
for i in lorry_data.keys():
    # set j to the the number of lorries at key 'i'
    for j in range(0, len(lorry_data[i])):
        # append each lorry in lorry_data to lorry list
        lorry.append(pd.DataFrame(lorry_data[i][j], index=[k]))
        # accumulate index
        k += 1

# lorry dataframe
lorry_df = pd.concat(lorry)

In [ ]:
# show first 5 rows of lorry_df
lorry_df.head()

In [ ]:
# list of depot locations (where nodes == depot)
depot_locations = np.where(location_df.is_depot)[0]

# list of customer locations (where nodes == customers)
customer_locations = np.where(location_df.is_customer)[0]

imports + visualising the map

In [ ]:
# imports
import networkx as nx
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt 

# parwise distance calculation for each node
euclidean = squareform(pdist(location_df[['x', 'y']]))

# edges list initialisation
edges = []

# loop through links_df rows
for _, (i, j) in links_df.iterrows():
    # append node at i, node at j, and their pairwise distance to edges
    edges.append((i, j, euclidean[i, j]))

# pos dict intialisation
pos = {}

# loop through location_df rows
for k, v in location_df[['x', 'y']].iterrows():
    # update pos dict with array of k, v 
    pos.update({k:v.values})

# initialise depot_labels dict
depot_labels = {}

# loop throgugh depot_locations
for i in depot_locations:
    # update depot_labels dict with {i:i}
    depot_labels.update({i:i})

# initialise customer_labels dict
customer_labels = {}

# loop through customer_locations
for i in customer_locations:
    # update customer_labels dict with {i:i}
    customer_labels.update({i:i})

# initialise nx Graph
G = nx.Graph()

# feed node list to G
G.add_nodes_from(location_df['id'].to_numpy())

# feed edges list to G
G.add_weighted_edges_from(edges)

# resize figure 
plt.figure(figsize=(16, 8))

# sketch graph
nx.draw(G, pos=pos, node_size=40)

# label depot nodes
nx.draw_networkx_labels(G, pos, depot_labels)

# label customer nodes
nx.draw_networkx_labels(G, pos, customer_labels)

# mark depot nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=depot_locations, node_color='r', node_size=400, alpha=0.9)

# mark customer nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=customer_locations, node_color='g', node_size=200, alpha=0.3)

clustering nodes (customer_locations) via nearest neighbour sorting

In [ ]:
# intialise cluster dict
cluster = {124:[], 127:[], 167:[], 523:[]}

# intialise nodes list
nodes = [] 

# loop through each node in customer_locations
for node in customer_locations:
    # check if node in nodes
    if node not in nodes:
        # initialise dist list
        dist = []
        # loop through each depot key
        for depot in cluster.keys():
            # append euclidean weights to dist 
            dist.append(euclidean[node, depot])
        # get shortest distance
        shortestDist = min(dist)
        # match shortest distance to equivalent node index
        nearestDepotIndex = np.where(euclidean[node]==shortestDist)
        # add node to relative nearest depot location
        cluster[int(nearestDepotIndex[0])].append(node)
        # track applied nodes
        nodes.append(node)
        # clear dist
        dist.clear()

# print allocated nodes to relative cluster points (depot locations)
print(cluster)

# clear nodes list
nodes.clear()

states and constraints

In [ ]:
# setting required column 
location_df['required'] = location_df['capacity']-location_df['level']

# displaying rows where is_customer true
location_df[location_df['is_customer']==True].head()

In [ ]:
# function: finding next nearest customer node
def nearest_customer(currentState, customerList):
    #initialise dist dict
    dist = {}
    
    # loop through customerList
    for i in customerList:
        # check for all where customer != currentState
        if i != currentState:
            # update dist with available customer index and their relative weights
            dist.update({i:euclidean[i,currentState]})

    # initialise temp list
    temp = []

    # loop through keys of dist 
    for i in dist.keys():
        # add weights to temp
        temp.append(dist[i])
    
    # get lowest weight
    _shortestDist = min(temp)
    
    # find relative index of lowest weight
    nearestCustomerIndex = np.where(euclidean[currentState]==_shortestDist)
    
    # return next index with relative weight
    return int(nearestCustomerIndex[0]), _shortestDist

In [ ]:
# test nearest_customer()
nearest_customer(1, [2, 3, 1, 5])

In [ ]:
# function: finding the nearest depot
def nearest_depot(currentState, depotList):
    # intialise dist dict
    dist = {}

    # loop through depotList
    for i in depotList:
        # update dist with depot and their relative distance values
        dist.update({i:euclidean[currentState, i]})
    
    # intialise temp list
    temp = []

    # loop through dist.keys()
    for i in dist.keys():
        # add values of each dist.keys() to temp 
        temp.append(dist[i])

    # get lowest depot weight
    _shortestDist = min(temp)

    # get the relative node index of the closest depot
    _nearestDepotIndex = np.where(euclidean[currentState]==_shortestDist)

    # return next depot index with relative weight
    return int(_nearestDepotIndex[0]), _shortestDist

In [ ]:
# test near_depot()
nearest_depot(1, depot_locations)

In [ ]:
#for i in lorry_df.iterrows():
lorry_df['depot'] = lorry_df.lorry_id.apply(lambda x: x.split('-')[0])
lorry_df.head()

greedy search (breadth-first-search)

In [ ]:
# intialise routes dict
routes = {}

# loop through lorry_df.index 
for i in lorry_df.index:
    # update routes with key: lorry_id, value: capacity
    routes.update({lorry_df['lorry_id'][i]:[(lorry_df.depot[i]),lorry_df['capacity'][i]]})

In [ ]:
# display journeys so far
routes

In [ ]:
# Routing from A to B using recursive Breadth-First-Search based algorithm to find shortest route, with back tracing
def routing(currState, toState, edges):
    # intialise visitedState list for tracking node traversal
    visitedState = [currState]
    # initialise visitedEdge list for tracking edge traversal
    visitedEdge = []
    # intialise edgeMemory list for storing the explored edges
    edgeMemory = []
    # intialise queue list for choosing the central node for next traversal
    queue = [currState]

    # loop while toState is not found
    while currState != toState:
        # remove and store the last element of the queue list as q
        q = queue.pop(0)
        
        # intialise currEdges dict which holds the next set of edges for traversals
        currEdges = {}
        # get the nodes at each edge, where either nodes are equivalent to q
        for edge in list(np.where(links_df[['id1', 'id2']]==q)[0]):
            # update the dict with the relative edge key and the node pairs
            currEdges.update({edge:[edges[edge][0], edges[edge][1]]})
        
        # store the explored edges in edgeMemory list
        edgeMemory.append(currEdges)

        # loop through each edge in currEdges
        for edge in currEdges:
            # check if the edge has been visited 
            if edge not in visitedEdge:
                # if not visited then add the edge to visitedEdge
                visitedEdge.append(edge)
                # check the node index in the edge that has not been visited 
                if currEdges[edge][0] not in visitedState and currEdges[edge][1] in visitedState:
                    # set currState to the unvisited node 
                    currState = currEdges[edge][0]
                    # mark the node in currState as visited 
                    visitedState.append(currState)
                    # add new currState to queue 
                    queue.append(currState)
                    # check if toState reached
                    if currState == toState:
                        # set currState to toState
                        currState = toState
                        # end loop
                        break 
                # similar to above but in the context of different index position of the node that has not been visited
                if currEdges[edge][1] not in visitedState and edges[edge][0] in visitedState:
                    currState = currEdges[edge][1]
                    visitedState.append(currState)
                    queue.append(currState)
                    if currState == toState:
                        currState = toState
                        break
    
    # set startState as the first node in visitedState list
    startState = visitedState[0]
    # set lastQ as the toState for tracking q node from end of order
    lastQ = [toState]
    # intialise backtrace list for backtracing the edges from edgeMemory
    backtrace = []
    # initialise nodetrace list for backtracing the nodes from edgeMemory
    nodetrace = []

    # intialise edgeMemoryReversed for reordering edgeMemory 
    edgeMemoryReversed = []
    # loop through each index between range 0 and length of edgeMemory
    for i in range (0, len(edgeMemory)):
        # set endElement to the last element in edgeMemory
        endElement = edgeMemory.pop(-1)
        # add the endElement to edgeMemoryReversed
        edgeMemoryReversed.append(endElement)
    
    # while last element in lastQ is not equivalent to the startState
    while lastQ[-1] != startState:
        # loop through each edge options in edgeMemoryReversed
        for edgeOpt in edgeMemoryReversed:
            # loop through each edge from as keys of the edge options
            for edge in edgeOpt.keys():

                # check if last element of lastQ is in the set of edge options given the edge
                if lastQ[-1] in edgeOpt[edge]:
                    # add the edge to backtrace 
                    backtrace.append(edge)
                    # check index of node which matches the lastQ element 
                    if lastQ[-1] == edgeOpt[edge][0] and lastQ[-1] != edgeOpt[edge][1]:
                        # update lastQ as the the node which does not match the lastQ element
                        lastQ.append(edgeOpt[edge][1])
                        # add the node to nodetrace
                        nodetrace.append(edgeOpt[edge][1])
                        # return to while iterate
                        break
                        # similar to above but in the context of different index postion of the matching node with lastQ element
                    if lastQ[-1] != edgeOpt[edge][0] and lastQ[-1] == edgeOpt[edge][1]:
                        lastQ.append(edgeOpt[edge][0])
                        nodetrace.append(edgeOpt[edge][0])
                        break
    
    # re-ordering edges from start to end
    edgeTraversed = []
    for i in range(0, len(backtrace)):
        endElement = backtrace.pop(-1)
        edgeTraversed.append(endElement)

    # re-ordering nodes from start to end 
    nodeOrder = []
    for i in range(0, len(nodetrace)):
        endElement = nodetrace.pop(-1)
        nodeOrder.append(endElement)
    
    routeWeight = []
    for edge in edgeTraversed:
        routeWeight.append(edges[edge][2])

    # return the the order in which nodes were visited and the order in which edges were traversed
    return nodeOrder, edgeTraversed, routeWeight

In [ ]:
# test: gives the node order in routing
print(f'node order: {routing(124, 10, edges)[0]}')
# test: gives the traversed edges in routing
print(f'edge order: {routing(124, 10, edges)[1]}')
# test: gives the weight routes for traversed
print(f'route weight: {routing(124, 10, edges)[2]}')

In [ ]:
# print(routes)
# temp variable initialisation for testing
tempRoutes = {'124-0': [['124', 8]], '124-1': [['124', 8]], '124-2': [['124', 16]], '124-3': [['124', 25]], '127-0': [['127', 8]], '127-1': [['127', 16]], '127-2': [['127', 25]], '167-0': [['167', 8]], '167-1': [['167', 16]], '167-2': [['167', 16]], '167-3': [['167', 25]], '523-0': [['523', 8]], '523-1': [['523', 16]], '523-2': [['523', 25]]}
tempCluster = {124: [10, 15, 37, 43, 71, 87, 129, 130, 140, 149, 170, 196, 209, 210, 252, 254, 263, 264, 266, 278, 281, 288, 297, 321, 327, 336, 369, 418, 461, 470, 476, 492, 503, 542, 561, 566, 572, 606, 616, 626, 627, 633], 127: [26, 77, 96, 126, 151, 178, 198, 215, 269, 279, 302, 357, 387, 416, 471, 515, 532, 534, 562, 583, 586, 609], 167: [20, 22, 25, 64, 72, 75, 86, 141, 144, 155, 166, 190, 193, 243, 282, 313, 316, 319, 348, 356, 381, 390, 393, 398, 431, 454, 466, 468, 469, 478, 485, 489, 513, 514, 548, 569, 580, 595, 615, 621], 523: [7, 152, 176, 235, 253, 276, 332, 338, 367, 401, 456, 490, 497, 508, 564, 588, 630]}

In [ ]:
# intialise totalRouteCost list to store route costs
routeTotalCost = []

In [ ]:
def route(lorryId, allocatedCustomers):
    currentId = lorryId
    operatingDepot = currentId.split('-')[0]
    currentCapacity = float(lorry_df[lorry_df['lorry_id']==currentId].capacity)
    currentCpm = float(lorry_df[lorry_df['lorry_id']==currentId].cpm)
    currentCptm = float(lorry_df[lorry_df['lorry_id']==currentId].cptm)
    currentState = int(lorry_df[lorry_df['lorry_id']==currentId].depot)
    currentDepot = int(lorry_df[lorry_df['lorry_id']==currentId].depot)
    nextCustomerCapacity = 0

    while (currentCapacity >= nextCustomerCapacity) and (len(allocatedCustomers)>0):
        nextCustomer = nearest_customer(currentState, allocatedCustomers)
        nextCustomerCapacity = location_df[location_df['id']==nextCustomer[0]]['capacity'].iloc[0]
        nextCustomerRequired = location_df[location_df['id']==nextCustomer[0]]['required'].iloc[0]
        nextCustomerTankSpace = nextCustomerCapacity-nextCustomerRequired

        # check if customer tank space is less than 50% 
        if nextCustomerTankSpace < (nextCustomerCapacity/2):
            #routes[currentId].append([int(nextCustomer[0]), -nextCustomerRequired])
            tempRoutes[currentId].append([int(nextCustomer[0]), -nextCustomerRequired])
            routeCost = round(sum(routing(currentState, nextCustomer[0], edges)[2])*(currentCpm+(currentCapacity*currentCptm)), 2)
            routeTotalCost.append(routeCost)
            currentCapacity = currentCapacity-nextCustomerRequired
            allocatedCustomers.remove(nextCustomer[0])
            currentState = nextCustomer[0]

        # check if customer tank space is more than 50%
        elif nextCustomerTankSpace > (nextCustomerCapacity/2):
            # currentState = nextCustomer[0]
            tempRoutes[currentId].append([int(nextCustomer[0]), -0])
            routeCost = round(sum(routing(currentState, nextCustomer[0], edges)[2])*(currentCpm+(currentCapacity*currentCptm)), 2)
            routeTotalCost.append(routeCost)
            currentCapacity = currentCapacity-0
            # if tank space is less than 50% then remove customer node as they do not require filling
            allocatedCustomers.remove(nextCustomer[0])
            currentState = nextCustomer[0]

        if len(allocatedCustomers) != 0:
            nextCustomerCapacity = location_df[location_df['id']==(nearest_customer(nextCustomer[0], allocatedCustomers)[0])]['required'].iloc[0]
    
    if currentCapacity < nextCustomerCapacity:
        tempCluster[currentDepot]=allocatedCustomers
        print(f'{currentId} finished operating at {currentState}')
    
    elif len(allocatedCustomers)==0:
        print(f'{currentId} finished operating at {currentState}')

# route("124-0", tempCluster[124])

In [ ]:
# for depot in cluster.keys():
#for depot in tempCluster.keys():
for i in depot_locations:
    #df = lorry_df[lorry_df['depot']==depot].sort_values(by=['capacity'])
    #df = lorry_df[lorry_df['depot']==i]
    df = lorry_df[lorry_df['depot']==f'{i}']
    #print(f'{df}\n')
    count = 0 

    for j in df.index:
        print(f'{df.loc[j].lorry_id} started operating')
        #route(df.loc[j].lorry_id, cluster[depot])
        #route(df.loc[j].lorry_id, tempCluster[depot])
        route(df.loc[j].lorry_id, tempCluster[i])
        count += 1
        
        #if len(cluster[depot])!=0 and count==len(df.index): 
        if len(tempCluster[i])!=0 and count==(len(df.index)):
            print(f'if statement reached')
            #while len(cluster[depot]!=0):
            while len(tempCluster[i]!=0):
                print(f'while statement reached')
                currentId = lorry_df.loc[j]['lorry_id']
                currentState = routes[currentId][-1][0]
                nearDepot = nearest_depot(currentState)
                print(f'{currentId} is refilling tank at {nearDepot[0]}')
                routeCost = round(sum(routing(currentState, nearDepot[0], edges)[2])*((lorry_df.loc[j]['cpm'])+(lorry_df.loc[j]['capacity']*lorry_df.loc[j]['cptm'])), 2)
                routeTotalCost.append(routeCost)
                #routes[currentId].append([int(nearDepot[0]),int(lorry_df.loc[j]['capacity'])])
                tempRoutes[currentId].append([int(nearDepot[0]),int(lorry_df.loc[j]['capacity'])])
                currentState = nearDepot[0]
                #route(currentId, cluster[depot])
                route(currentId, tempCluster[i])
        #elif len(cluster[depot])==0:
        elif len(tempCluster[i])==0:
            break



In [ ]:
#print(f'tempRoutes: {tempRoutes} \n')
#print(f'tempCluster: {tempCluster}')
#print(lorry_df)
for i in tempRoutes.keys():
    print(i)
    print(f'{tempRoutes[i]}\n')

In [ ]:
for i in depot_locations:
    print(i)
    df = lorry_df[lorry_df['depot']==f'{i}']
    print(df)
    print(f'\n')